<h1><center>Pivoting y Grouping Sets

<h2>Parte 01. Escribiendo consultas con PIVOT Y UNPIVOT

1.1. Abrir la base de datos TSQL.

In [2]:
USE TSQL;
GO

Commands completed successfully.

Total execution time: 00:00:00

1.2. Crear un vista de categorías de productos, sus cantidaddes por año.

In [3]:
IF OBJECT_ID('Sales.CategoryQtyYear', 'V') IS NOT NULL DROP VIEW Sales.CategoryQtyYear
GO
CREATE VIEW Sales.CategoryQtyYear
AS
SELECT c.categoryname AS Category,
       SUM(od.qty) AS TotalQty, -- Se suma la cantidad
       YEAR(o.orderdate) AS Orderyear
FROM Production.Categories AS c
INNER JOIN Production.Products AS p ON c.categoryid = p.categoryid
INNER JOIN Sales.OrderDetails AS od ON p.productid = od.productid
INNER JOIN Sales.Orders AS o ON od.orderid = o.orderid
GROUP BY c.categoryname, YEAR(o.orderdate); -- Se agrupa por categoría y año
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.002

1.3. Probar la vista creada.

In [5]:
SELECT Category, TotalQty, Orderyear
FROM Sales.CategoryQtyYear;

(24 rows affected)

Total execution time: 00:00:00.055

Category,TotalQty,Orderyear
Meat/Poultry,950,2006
Seafood,2716,2008
Dairy Products,4374,2007
Grains/Cereals,1377,2008
Confections,4137,2007
Produce,858,2008
Beverages,1842,2006
Condiments,1441,2008
Dairy Products,2689,2008
Seafood,3679,2007


1.4. Utilizar PIVOT entre categorías y años de ordenes.

In [6]:
SELECT Category, [2006], [2007], [2008]
FROM (SELECT Category, TotalQty, Orderyear FROM Sales.CategoryQtyYear) AS D -- Cambiado Qty a TotalQty
PIVOT(SUM(TotalQty) FOR Orderyear IN ([2006], [2007], [2008])) AS pvt -- Cambiado QTY a TotalQty
ORDER BY Category;

(8 rows affected)

Total execution time: 00:00:00.016

Category,2006,2007,2008
Beverages,1842,3996,3694
Condiments,962,2895,1441
Confections,1357,4137,2412
Dairy Products,2086,4374,2689
Grains/Cereals,549,2636,1377
Meat/Poultry,950,2189,1060
Produce,549,1583,858
Seafood,1286,3679,2716


1.5. Para utilizar UNPIVOT, crear una tabla con el resultado de la consulta con PIVOT anterior.

In [7]:
CREATE TABLE [Sales].[PivotedCategorySales](
    [Category] [nvarchar](15) NOT NULL,
    [2006] [int] NULL,
    [2007] [int] NULL,
    [2008] [int] NULL);
GO

INSERT INTO Sales.PivotedCategorySales (Category, [2006], [2007], [2008])
SELECT Category, [2006], [2007], [2008]
FROM (SELECT Category, TotalQty, Orderyear FROM Sales.CategoryQtyYear) AS D -- Cambiado Qty a TotalQty
PIVOT(SUM(TotalQty) FOR Orderyear IN ([2006], [2007], [2008])) AS p; -- Cambiado QTY a TotalQty
GO

Commands completed successfully.

(8 rows affected)

Total execution time: 00:00:00.010

1.6. Probrar la tabla generada

In [8]:
SELECT Category, [2006],[2007],[2008]
FROM Sales.PivotedCategorySales;

(8 rows affected)

Total execution time: 00:00:00.003

Category,2006,2007,2008
Beverages,1842,3996,3694
Condiments,962,2895,1441
Confections,1357,4137,2412
Dairy Products,2086,4374,2689
Grains/Cereals,549,2636,1377
Meat/Poultry,950,2189,1060
Produce,549,1583,858
Seafood,1286,3679,2716


1.7. Utilizar UNPIVOT.

In [9]:
SELECT category, qty, orderyear
FROM Sales.PivotedCategorySales
UNPIVOT(qty FOR orderyear IN([2006],[2007],[2008])) AS unpvt;

(24 rows affected)

Total execution time: 00:00:00.013

category,qty,orderyear
Beverages,1842,2006
Beverages,3996,2007
Beverages,3694,2008
Condiments,962,2006
Condiments,2895,2007
Condiments,1441,2008
Confections,1357,2006
Confections,4137,2007
Confections,2412,2008
Dairy Products,2086,2006


1.8. Limpiar los cambios realizados.


In [10]:
IF OBJECT_ID('Sales.CategoryQtyYear','V') IS NOT NULL DROP VIEW Sales.CategoryQtyYear
IF OBJECT_ID('Sales.PivotedCategorySales') IS NOT NULL DROP TABLE Sales.PivotedCategorySales
GO

Commands completed successfully.

Total execution time: 00:00:00.010

<h2>Parte 02. Trabajando con GROUPING SETS

2.1. Ejecutar la siguiente vistas

In [11]:
-- Step 2: Setup objects for demo
IF OBJECT_ID('Sales.CategorySales','V') IS NOT NULL DROP VIEW Sales.CategorySales
GO
CREATE VIEW Sales.CategorySales
AS
SELECT  c.categoryname AS Category,
        o.empid AS Emp,
        o.custid AS Cust,
        od.qty AS Qty,
        YEAR(o.orderdate) AS Orderyear
FROM    Production.Categories AS c
        INNER JOIN Production.Products AS p ON c.categoryid=p.categoryid
        INNER JOIN Sales.OrderDetails AS od ON p.productid=od.productid
        INNER JOIN Sales.Orders AS o ON od.orderid=o.orderid
WHERE c.categoryid IN (1,2,3) AND o.custid BETWEEN 1 AND 5; --limits results for slides
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.002

2.2. Elaborar una consulta sin utilizar GROUPING SETS.

In [12]:
SELECT Category, NULL AS Cust, SUM(Qty) AS TotalQty
FROM Sales.CategorySales
GROUP BY category
UNION ALL 
SELECT  NULL, Cust, SUM(Qty) AS TotalQty
FROM Sales.CategorySales
GROUP BY cust 
UNION ALL
SELECT NULL, NULL, SUM(Qty) AS TotalQty
FROM Sales.CategorySales;

(9 rows affected)

Total execution time: 00:00:00.020

Category,Cust,TotalQty
Beverages,NULL,513
Condiments,NULL,114
Confections,NULL,372
NULL,1,80
NULL,2,12
NULL,3,154
NULL,4,241
NULL,5,512
NULL,NULL,999


2.3. Consultar con GROUPING SETS.

In [13]:
SELECT Category, Cust, SUM(Qty) AS TotalQty
FROM Sales.CategorySales
GROUP BY 
GROUPING SETS((Category),(Cust),())
ORDER BY Category DESC, Cust desc;

(9 rows affected)

Total execution time: 00:00:00.020

Category,Cust,TotalQty
Confections,NULL,372
Condiments,NULL,114
Beverages,NULL,513
NULL,5,512
NULL,4,241
NULL,3,154
NULL,2,12
NULL,1,80
NULL,NULL,999


2.4. Consultar con CUBE.

In [14]:
SELECT Category, Cust, SUM(Qty) AS TotalQty
FROM Sales.CategorySales
GROUP BY CUBE(Category,Cust)
ORDER BY Category, Cust; 

(21 rows affected)

Total execution time: 00:00:00.015

Category,Cust,TotalQty
NULL,NULL,999
NULL,1,80
NULL,2,12
NULL,3,154
NULL,4,241
NULL,5,512
Beverages,NULL,513
Beverages,1,36
Beverages,2,5
Beverages,3,105


2.5. Consultar con ROLLUP

In [15]:
SELECT Category, Cust, SUM(Qty) AS TotalQty
FROM Sales.CategorySales
GROUP BY ROLLUP(Category,Cust)
ORDER BY Category, Cust;

(16 rows affected)

Total execution time: 00:00:00.010

Category,Cust,TotalQty
NULL,NULL,999
Beverages,NULL,513
Beverages,1,36
Beverages,2,5
Beverages,3,105
Beverages,4,112
Beverages,5,255
Condiments,NULL,114
Condiments,1,44
Condiments,3,4


2.6. Utilizar Grouping_ID.

In [16]:
SELECT	GROUPING_ID(Category)AS grpCat, GROUPING_ID(Cust) AS grpCust, 
		Category, Cust, SUM(Qty) AS TotalQty
FROM Sales.CategorySales
GROUP BY CUBE(Category,Cust)
ORDER BY Category, Cust;

(21 rows affected)

Total execution time: 00:00:00.016

grpCat,grpCust,Category,Cust,TotalQty
1,1,NULL,NULL,999
1,0,NULL,1,80
1,0,NULL,2,12
1,0,NULL,3,154
1,0,NULL,4,241
1,0,NULL,5,512
0,1,Beverages,NULL,513
0,0,Beverages,1,36
0,0,Beverages,2,5
0,0,Beverages,3,105


2.7. Finalmente limpiar los cambios

In [17]:
IF OBJECT_ID('Sales.CategorySales','V') IS NOT NULL DROP VIEW Sales.CategorySales
GO

Commands completed successfully.

Total execution time: 00:00:00.003